In [9]:
import pandas as pd
import numpy as np
import os
import datetime

In [10]:
import git
import sys
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir

In [11]:
pop = pd.read_csv(f"{homedir}/data/us/demographics/county_populations.csv")
submission = pd.read_csv(f"{homedir}/models/Submissions/processing/formatted_submission.csv")

In [12]:
submission['Date'] = submission.id.str.rsplit("-", n = 1, expand = True)[0]
submission['FIPS'] = submission.id.str.rsplit("-", n = 1, expand = True)[1].astype(int)
submission = submission.join(pop.set_index('FIPS'), on='FIPS')
submission['total_pop'] = submission['total_pop'].fillna(2000)
submission['total_pop'] = submission['total_pop'].astype(int)

In [13]:
temp = submission.copy()
Large_10 = temp.loc[submission['10'] >= 0.075*submission['total_pop']]
temp.loc[Large_10.index.values,'10'] = Large_10.total_pop.values*0.075
temp.loc[Large_10.index.values,'20'] = Large_10.total_pop.values*(1.1)**1*0.075
temp.loc[Large_10.index.values,'30'] = Large_10.total_pop.values*(1.1)**2*0.075
temp.loc[Large_10.index.values,'40'] = Large_10.total_pop.values*(1.1)**3*0.075
temp.loc[Large_10.index.values,'50'] = Large_10.total_pop.values*(1.1)**4*0.075
temp.loc[Large_10.index.values,'60'] = Large_10.total_pop.values*(1.1)**5*0.075
temp.loc[Large_10.index.values,'70'] = Large_10.total_pop.values*(1.1)**6*0.075
temp.loc[Large_10.index.values,'80'] = Large_10.total_pop.values*(1.1)**7*0.075
temp.loc[Large_10.index.values,'90'] = Large_10.total_pop.values*(1.1)**8*0.075

In [14]:
Large_Rat = temp.loc[temp['90']/temp['10'] >= 10]
Meanval = (temp.loc[Large_Rat.index.values,'10'] + temp.loc[Large_Rat.index.values,'20'] + temp.loc[Large_Rat.index.values,'30'])/3
temp.loc[Large_Rat.index.values,'10'] = Meanval/(1.2)
temp.loc[Large_Rat.index.values,'20'] = Meanval*(1.2)**0
temp.loc[Large_Rat.index.values,'30'] = Meanval*(1.2)**1
temp.loc[Large_Rat.index.values,'40'] = Meanval*(1.2)**2
temp.loc[Large_Rat.index.values,'50'] = Meanval*(1.2)**3
temp.loc[Large_Rat.index.values,'60'] = Meanval*(1.2)**4
temp.loc[Large_Rat.index.values,'70'] = Meanval*(1.2)**5
temp.loc[Large_Rat.index.values,'80'] = Meanval*(1.2)**6
temp.loc[Large_Rat.index.values,'90'] = Meanval*(1.2)**7

In [15]:
temp = temp[['id','10','20','30','40','50','60','70','80','90']]

In [16]:
temp.to_csv('submission_modified.csv')